In [1]:
import h5py
import pandas as pd
import numpy as np
import os

In [2]:
!pwd

/home/jovyan/workspace/Desktop/bsc_outputs/7_aillades_2intent


In [3]:
def populate_tables(data_folder_name='./data', 
                   tracers=[1,2,3,4,5,6,7]):
    #########################################
    """
    This function create the txt files with the strain rate data
    
    Arguments:
    - data_folder_name (str): folder where the raw h5 files are stored. 
        Default is 'outputs-fixed-values'.
    - tracers (str): name of tracers. Default is 'lag'.
    
    Returns:
    - Folder called txt_outputs with all the files.
    """
    #########################################
    
    # Check if the data folder exists
    if not os.path.exists(data_folder_name):
        raise FileNotFoundError(f"Folder '{data_folder_name}' not found.")
    
    # count number of times, checking the number that files that starts with prefix
    prefix = f'tip{tracers[0]}_velocitygradient'
    files = os.listdir(data_folder_name)
    num_files = sum([1 for file in files if file.startswith(prefix)])
    print(num_files)
    # loop by number of files
    time = 0
    #print('num files:', num_files)
    while time < num_files:
        print(f'time: {time}')
        #########################################
        #  LOOP by tracer #
        #########################################
        # create a variable name to store coord
        dataset_name = f'np_array_{time}'
        globals()[dataset_name] = []
        #print('dataset_name', dataset_name)
        for tracer in tracers:
            ## COORDINATES ##
            # open Coordinates H5 
            h5_file_coord =  f'./{data_folder_name}/tip{tracer}-{time}.h5'
            if not os.path.exists(h5_file_coord):
                    print(f"Warning: H5 file not found at '{h5_file_coord}'. Skipping this time step.")
                    time += 1222
                    continue

            with h5py.File(h5_file_coord, 'r') as file:
                for key in file.keys():
                    # create a string with the name of the var, and assign numpy value from h5
                    globals()[dataset_name].append(file[key][:][0])
                    #print('ds_name',globals()[dataset_name])
                

        # create a table for each time and populate them with numpy array 
        # first create the name of the table
        df_name_by_t = f'df_t_{time}'
        # create the table
        globals()[df_name_by_t] = pd.DataFrame(globals()[dataset_name], columns=['x', 'y', 'z'])
        #add index in first column
        globals()[df_name_by_t].insert(0, 'index', globals()[df_name_by_t].index)
        #prit('df_name_by_t', globals()[df_name_by_t])
        

        ## Velocity Gradient TENSOR ##
        # create a variable name for storing strain rate tensor array
        dataset_name = f'np_array_{time}'
        globals()[dataset_name] = []
        for tracer in tracers:
            # open the Strain Rate Tensor H5 files 
            h5_file_path =  f'./{data_folder_name}/tip{tracer}_velocitygradient-{time}.h5'
            if not os.path.exists(h5_file_path):
                    print(f"Warning: H5 Strain Rate Tensor file not found at '{h5_file_path}'. Skipping this time step.")
                    time += 1
                    continue

            with h5py.File(h5_file_path, 'r') as file:
                for key in file.keys():
                    globals()[dataset_name].append(file[key][:][0])
                    print('ds_name_velgrad',globals()[dataset_name])
                    print('ds_name_velgrad[0]',globals()[dataset_name][0])
                    print('testing')
                    print(globals()[dataset_name][0][0])

            # [a00,a01,a02,a10,a11,a12,a20,a21,a22]
            print('tracer',tracer)
            tensor_index = tracer - 1
            globals()[df_name_by_t].loc[tensor_index, 'VG00'] = globals()[dataset_name][tensor_index][0]
            globals()[df_name_by_t].loc[tensor_index, 'VG01'] = globals()[dataset_name][tensor_index][1]
            globals()[df_name_by_t].loc[tensor_index, 'VG02'] = globals()[dataset_name][tensor_index][2]
            globals()[df_name_by_t].loc[tensor_index, 'VG10'] = globals()[dataset_name][tensor_index][3]
            globals()[df_name_by_t].loc[tensor_index, 'VG11'] = globals()[dataset_name][tensor_index][4]
            globals()[df_name_by_t].loc[tensor_index, 'VG12'] = globals()[dataset_name][tensor_index][5]
            globals()[df_name_by_t].loc[tensor_index, 'VG20'] = globals()[dataset_name][tensor_index][6]
            globals()[df_name_by_t].loc[tensor_index, 'VG21'] = globals()[dataset_name][tensor_index][7]
            globals()[df_name_by_t].loc[tensor_index, 'VG22'] = globals()[dataset_name][tensor_index][8]
            print('fi')
     
            
            #print('Time',time)
            #print('globals()[df_name_by_t]', globals()[df_name_by_t])
        ## save the df as txt
        output_folder = 'txt_outputs_test'
        output_file = os.path.join(output_folder, f'results{df_name_by_t}.txt')

        # check if the folder exists; if not, create it
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
            print('out folder: ', output_folder)

        # save to tab-separated values (.txt) file
        globals()[df_name_by_t].to_csv(output_file, sep='\t', index=False)

        #########


        print(f'hi {df_name_by_t}') 
        print(globals()[df_name_by_t].tail())
        time += 1
        
            
        

In [4]:
populate_tables()

341
time: 0
ds_name_velgrad [array([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)]
ds_name_velgrad[0] [0. 0. 0. 0. 0. 0. 0. 0. 0.]
testing
0.0
tracer 1
fi
ds_name_velgrad [array([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)]
ds_name_velgrad[0] [0. 0. 0. 0. 0. 0. 0. 0. 0.]
testing
0.0
tracer 2
fi
ds_name_velgrad [array([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)]
ds_name_velgrad[0] [0. 0. 0. 0. 0. 0. 0. 0. 0.]
testing
0.0
tracer 3
fi
ds_name_velgrad [array([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)]
ds_name_velgrad[0] [0. 0. 0. 0. 0. 0. 0. 0. 0.]
testing
0.0
tracer 4
fi
ds_name_velgrad [array([0., 0., 

6  1.721033e-16  2.770015e-18  
time: 11
ds_name_velgrad [array([ 3.2439952e-17,  1.8999932e-16,  6.5637780e-16,  1.1653631e-16,
        2.9924446e-17,  1.6858286e-16,  2.3934282e-16, -1.1639481e-16,
       -3.0533983e-17], dtype=float32)]
ds_name_velgrad[0] [ 3.2439952e-17  1.8999932e-16  6.5637780e-16  1.1653631e-16
  2.9924446e-17  1.6858286e-16  2.3934282e-16 -1.1639481e-16
 -3.0533983e-17]
testing
3.2439952e-17
tracer 1
fi
ds_name_velgrad [array([ 3.2439952e-17,  1.8999932e-16,  6.5637780e-16,  1.1653631e-16,
        2.9924446e-17,  1.6858286e-16,  2.3934282e-16, -1.1639481e-16,
       -3.0533983e-17], dtype=float32), array([ 7.0370313e-17,  8.3491938e-17,  9.7205300e-16, -8.0719891e-18,
       -2.4061508e-19,  1.5067952e-16,  7.3755500e-17, -4.2795558e-18,
        1.1730326e-17], dtype=float32)]
ds_name_velgrad[0] [ 3.2439952e-17  1.8999932e-16  6.5637780e-16  1.1653631e-16
  2.9924446e-17  1.6858286e-16  2.3934282e-16 -1.1639481e-16
 -3.0533983e-17]
testing
3.2439952e-17
tracer 

tracer 4
fi
ds_name_velgrad [array([-6.2099843e-17,  1.0297413e-16,  2.6739512e-16, -2.5904953e-17,
        4.1243338e-17,  4.8687537e-17,  7.9023297e-17, -1.7471844e-16,
       -1.9192812e-17], dtype=float32), array([ 9.5352133e-17, -2.4186870e-17,  5.3108378e-16,  1.1416532e-17,
       -1.4829206e-18, -7.3984225e-17, -1.4434700e-16,  5.1050076e-18,
       -7.5097094e-17], dtype=float32), array([ 7.4120842e-17, -5.6107674e-17,  2.1081586e-16,  1.5507448e-17,
        3.9225904e-17, -4.2270964e-17, -5.9840793e-17,  1.3177279e-18,
       -1.1397620e-16], dtype=float32), array([-2.6273774e-17, -3.5589483e-18, -1.4487417e-16, -3.2439507e-18,
       -3.2129938e-17,  8.1600638e-19,  1.1044383e-16,  1.0204020e-18,
        4.6081090e-17], dtype=float32), array([ 2.9708473e-17, -2.5551952e-17,  4.0659360e-16, -2.0753138e-17,
        5.5584726e-18, -6.1505557e-17,  6.6733557e-17, -9.7779562e-19,
       -5.3379649e-17], dtype=float32)]
ds_name_velgrad[0] [-6.2099843e-17  1.0297413e-16  2.6739512e

fi
ds_name_velgrad [array([-1.8037957e-16,  1.9509333e-16,  4.6462584e-16, -6.0612420e-17,
        1.3545476e-16,  7.9472721e-17,  9.7519373e-17, -3.1411748e-16,
        2.8360324e-17], dtype=float32), array([ 1.8938222e-16, -1.5258455e-17,  1.7898740e-15,  4.7490927e-17,
       -3.6787177e-17,  3.2785130e-16, -4.5103615e-16,  4.1367514e-17,
        8.4248286e-17], dtype=float32)]
ds_name_velgrad[0] [-1.8037957e-16  1.9509333e-16  4.6462584e-16 -6.0612420e-17
  1.3545476e-16  7.9472721e-17  9.7519373e-17 -3.1411748e-16
  2.8360324e-17]
testing
-1.8037957e-16
tracer 2
fi
ds_name_velgrad [array([-1.8037957e-16,  1.9509333e-16,  4.6462584e-16, -6.0612420e-17,
        1.3545476e-16,  7.9472721e-17,  9.7519373e-17, -3.1411748e-16,
        2.8360324e-17], dtype=float32), array([ 1.8938222e-16, -1.5258455e-17,  1.7898740e-15,  4.7490927e-17,
       -3.6787177e-17,  3.2785130e-16, -4.5103615e-16,  4.1367514e-17,
        8.4248286e-17], dtype=float32), array([ 1.5751119e-16,  7.2327951e-17,  3.

ds_name_velgrad [array([-1.9584476e-16,  2.0965407e-16,  5.6857955e-16, -9.3637123e-17,
        9.1676818e-17,  9.7519697e-17,  8.0872165e-17, -3.2309881e-16,
        6.5844027e-17], dtype=float32)]
ds_name_velgrad[0] [-1.9584476e-16  2.0965407e-16  5.6857955e-16 -9.3637123e-17
  9.1676818e-17  9.7519697e-17  8.0872165e-17 -3.2309881e-16
  6.5844027e-17]
testing
-1.9584476e-16
tracer 1
fi
ds_name_velgrad [array([-1.9584476e-16,  2.0965407e-16,  5.6857955e-16, -9.3637123e-17,
        9.1676818e-17,  9.7519697e-17,  8.0872165e-17, -3.2309881e-16,
        6.5844027e-17], dtype=float32), array([ 2.29484810e-16,  3.25929179e-17,  2.13987543e-15, -7.77088540e-17,
        3.99240443e-17, -1.09735534e-16, -3.08757882e-16, -3.47674599e-17,
        2.20797455e-16], dtype=float32)]
ds_name_velgrad[0] [-1.9584476e-16  2.0965407e-16  5.6857955e-16 -9.3637123e-17
  9.1676818e-17  9.7519697e-17  8.0872165e-17 -3.2309881e-16
  6.5844027e-17]
testing
-1.9584476e-16
tracer 2
fi
ds_name_velgrad [array([-

...

fi
ds_name_velgrad [array([ 5.2753937e-17,  7.9876255e-16,  2.4062687e-16, -1.4257989e-16,
       -1.2596471e-17, -3.9173332e-17, -3.1331259e-16, -4.6618788e-16,
        6.9319549e-17], dtype=float32), array([ 2.4126979e-16,  5.1190655e-18,  4.2055503e-16,  1.5134659e-17,
       -3.3559581e-17, -6.9832650e-19, -3.4258827e-17, -4.5888979e-17,
       -2.9872255e-16], dtype=float32)]
ds_name_velgrad[0] [ 5.2753937e-17  7.9876255e-16  2.4062687e-16 -1.4257989e-16
 -1.2596471e-17 -3.9173332e-17 -3.1331259e-16 -4.6618788e-16
  6.9319549e-17]
testing
5.2753937e-17
tracer 2
fi
ds_name_velgrad [array([ 5.2753937e-17,  7.9876255e-16,  2.4062687e-16, -1.4257989e-16,
       -1.2596471e-17, -3.9173332e-17, -3.1331259e-16, -4.6618788e-16,
        6.9319549e-17], dtype=float32), array([ 2.4126979e-16,  5.1190655e-18,  4.2055503e-16,  1.5134659e-17,
       -3.3559581e-17, -6.9832650e-19, -3.4258827e-17, -4.5888979e-17,
       -2.9872255e-16], dtype=float32), array([-1.0726626e-16,  1.3201685e-17,  1.8

tracer 1
fi
ds_name_velgrad [array([ 2.1460326e-16,  1.1378662e-15,  5.7323574e-16, -3.9324630e-17,
       -1.5934557e-16, -9.7967983e-17, -3.6650748e-16, -6.6586456e-16,
       -2.3236012e-16], dtype=float32), array([ 1.2056683e-16,  2.2945241e-19,  3.8510546e-16,  1.8562826e-17,
       -8.6446859e-18,  8.4173727e-17, -6.7515746e-17, -9.7971785e-18,
       -2.3183263e-16], dtype=float32)]
ds_name_velgrad[0] [ 2.1460326e-16  1.1378662e-15  5.7323574e-16 -3.9324630e-17
 -1.5934557e-16 -9.7967983e-17 -3.6650748e-16 -6.6586456e-16
 -2.3236012e-16]
testing
2.1460326e-16
tracer 2
fi
ds_name_velgrad [array([ 2.1460326e-16,  1.1378662e-15,  5.7323574e-16, -3.9324630e-17,
       -1.5934557e-16, -9.7967983e-17, -3.6650748e-16, -6.6586456e-16,
       -2.3236012e-16], dtype=float32), array([ 1.2056683e-16,  2.2945241e-19,  3.8510546e-16,  1.8562826e-17,
       -8.6446859e-18,  8.4173727e-17, -6.7515746e-17, -9.7971785e-18,
       -2.3183263e-16], dtype=float32), array([-7.7988798e-17,  2.5162249e

tracer 7
fi
hi df_t_314
   index            x            y           z          VG00          VG01  \
2      2   993.340617   921.028280 -550.752118 -4.985651e-17  6.959118e-18   
3      3  1703.202678   932.823875 -238.262119 -7.000295e-17  3.296364e-18   
4      4  1404.626667   919.341104 -486.346859  1.462949e-16 -5.155115e-17   
5      5  1704.753930  1195.250398 -241.634372 -7.198111e-17  3.047282e-18   
6      6  1524.584434  1721.621641 -449.637801  4.523911e-17 -5.931820e-16   

           VG02          VG10          VG11          VG12          VG20  \
2  2.109172e-16 -4.220040e-17 -1.044020e-17 -1.869242e-17  3.714107e-17   
3  8.267766e-18 -2.039849e-18 -3.385619e-17 -1.684715e-18  8.921485e-17   
4  3.087933e-16  1.286788e-17  5.176669e-17  5.166847e-17  9.919740e-18   
5  3.341817e-18  2.370736e-18 -2.602410e-17 -6.539291e-19  9.258383e-17   
6 -2.048941e-16 -5.607496e-17 -7.644526e-17  1.079934e-16 -2.575959e-16   

           VG21          VG22  
2 -4.076788e-17  5.52482

fi
ds_name_velgrad [array([ 1.4283255e-16,  8.0701006e-16,  1.8373706e-16,  2.3174194e-16,
       -4.8916245e-17,  4.6618602e-16, -3.5549944e-16, -8.1305205e-16,
        8.6987915e-17], dtype=float32), array([ 1.4548024e-16,  1.6333621e-17,  6.0482583e-16,  1.4028312e-17,
       -3.8159900e-17,  2.3718010e-18, -4.9479423e-17, -7.0537827e-18,
       -2.5866549e-16], dtype=float32)]
ds_name_velgrad[0] [ 1.4283255e-16  8.0701006e-16  1.8373706e-16  2.3174194e-16
 -4.8916245e-17  4.6618602e-16 -3.5549944e-16 -8.1305205e-16
  8.6987915e-17]
testing
1.4283255e-16
tracer 2
fi
ds_name_velgrad [array([ 1.4283255e-16,  8.0701006e-16,  1.8373706e-16,  2.3174194e-16,
       -4.8916245e-17,  4.6618602e-16, -3.5549944e-16, -8.1305205e-16,
        8.6987915e-17], dtype=float32), array([ 1.4548024e-16,  1.6333621e-17,  6.0482583e-16,  1.4028312e-17,
       -3.8159900e-17,  2.3718010e-18, -4.9479423e-17, -7.0537827e-18,
       -2.5866549e-16], dtype=float32), array([-4.1509624e-17,  2.2974110e-18,  2.5

tracer 2
fi
ds_name_velgrad [array([ 2.1502708e-16,  7.0472543e-16,  8.5545706e-16,  1.3255383e-16,
       -1.1332751e-17, -7.0474225e-18,  1.3259200e-16, -8.4700787e-17,
       -8.4345754e-18], dtype=float32), array([ 1.8575107e-16, -1.6594883e-17,  2.1877246e-16, -4.3926179e-17,
       -3.2135245e-17, -7.9675550e-18, -2.2122828e-17,  3.9417942e-17,
       -1.1230955e-16], dtype=float32), array([-6.0912833e-17, -2.5782579e-17,  3.2424059e-16, -5.4049516e-18,
       -1.6183016e-17,  1.5500033e-18, -8.1078814e-17,  9.0498635e-17,
        6.1433817e-17], dtype=float32)]
ds_name_velgrad[0] [ 2.1502708e-16  7.0472543e-16  8.5545706e-16  1.3255383e-16
 -1.1332751e-17 -7.0474225e-18  1.3259200e-16 -8.4700787e-17
 -8.4345754e-18]
testing
2.1502708e-16
tracer 3
fi
ds_name_velgrad [array([ 2.1502708e-16,  7.0472543e-16,  8.5545706e-16,  1.3255383e-16,
       -1.1332751e-17, -7.0474225e-18,  1.3259200e-16, -8.4700787e-17,
       -8.4345754e-18], dtype=float32), array([ 1.8575107e-16, -1.6594883e

In [8]:
h5_file_coord =  f'./outputs_3d_no_t_copy/lag_velocitygradient-1.h5'

with h5py.File(h5_file_coord, 'r') as file:
            # create a variable name for storing strain rate tensor array
            dataset_name = f'np_array_6'

            for key in file.keys():
                # create a string with the name of the var, and assign numpy value from h5
                 dataset_name = file[key][:] 
                    

dataset_name[0]

HDF5-DIAG: Error detected in HDF5 (1.12.1) thread 0:
  #000: H5F.c line 620 in H5Fopen(): unable to open file
    major: File accessibility
    minor: Unable to open file
  #001: H5VLcallback.c line 3501 in H5VL_file_open(): failed to iterate over available VOL connector plugins
    major: Virtual Object Layer
    minor: Iteration failed
  #002: H5PLpath.c line 578 in H5PL__path_table_iterate(): can't iterate over plugins in plugin path '(null)'
    major: Plugin for dynamically loaded library
    minor: Iteration failed
  #003: H5PLpath.c line 620 in H5PL__path_table_iterate_process_path(): can't open directory: /usr/local/hdf5/lib/plugin
    major: Plugin for dynamically loaded library
    minor: Can't open directory or file
  #004: H5VLcallback.c line 3351 in H5VL__file_open(): open failed
    major: Virtual Object Layer
    minor: Can't open object
  #005: H5VLnative_file.c line 97 in H5VL__native_file_open(): unable to open file
    major: File accessibility
    minor: Unable to o

RuntimeError: Unspecified error in H5Fopen (return value <0)